In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

# Лаба 1. Расчет рейтингов фильмов – RDD

#### Дедлайн

Воскресенье,  16 октября 2022 до 23:59:00(МСК).

#### Дедлайн Github

Среда, 19 октября 2022 до 23:59:00(МСК).

## Задача

По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

Задание необходимо сделать, используя RDD.

## Описание данных

Имеются следующие входные данные:

* Таблица `users x movies` с рейтингами. Архив с датасетом нужно скачать с сайта [GroupLens](http://files.grouplens.org/datasets/movielens/ml-100k.zip). Также он загружен на HDFS в `/labs/laba01/ml-100k`. Файл u.data содержит все оценки, а файл u.item — список всех фильмов.
* Информация о структуре датасета расположена [здесь](http://files.grouplens.org/datasets/movielens/ml-100k-README.txt).

`!hdfs dfs -ls /labs/laba01/ml-100k`

* `id фильма` для расчета индивидуальных характеристик — в Личном кабинете на странице [Лабы 1](https://lk-spark.newprolab.com/lab/slaba01).

## Результат

Выходной формат файла — json. Пример решения:

```json
{
   "hist_film": [  
      134,
      123,
      782,
      356,
      148
   ],
   "hist_all": [  
      134,
      123,
      782,
      356,
      148
   ]
}
```

В поле `“hist_film”` нужно указать для заданного `id` фильма количество поставленных оценок в следующем порядке: `"1", "2", "3", "4", "5"`. То есть, сколько было единичек, двоек, троек и т.д.

В поле `“hist_all”` нужно указать то же самое, только для всех фильмов общее количество поставленных оценок в том же порядке: `"1", "2", "3", "4", "5"`.

## Проверка

Файл необходимо положить в свою домашнюю директорию на кластере под названием: `lab01.json`.

Проверка осуществляется автоматическим скриптом на странице лабы в личном кабинете.

Обязательное условие зачета лабораторной работы – это выкладка после дедлайна лабы своего решения в репозиторий. Как это сделать, можно прочитать [здесь](/git.md). Если будут вопросы – спрашивайте в Telegram.

In [3]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [85]:
raw_rates = sc.textFile("/labs/laba01/ml-100k/u.data")
raw_films = sc.textFile("/labs/laba01/ml-100k/u.item")

In [86]:
raw_rates.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [87]:
raw_films.take(5)

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0']

In [109]:
rates_all = raw_rates.map(lambda x: (x.split("\t")[2], 1))\
                    .countByKey()\
                    .items()

rates_96 = raw_rates.map(lambda x: x.split("\t"))\
                    .filter(lambda x: x[1]=="96")\
                    .map(lambda x: x[2], 1)\
                    .countByKey()\
                    .items()

films = raw_films.map(lambda x: x.split('|'))

In [110]:
rates_96

dict_items([('5', 103), ('1', 6), ('4', 123), ('3', 43), ('2', 20)])

In [111]:
dict(sorted(rates_96))

{'1': 6, '2': 20, '3': 43, '4': 123, '5': 103}

In [112]:
dict(sorted(rates_all))

{'1': 6110, '2': 11370, '3': 27145, '4': 34174, '5': 21201}

In [124]:
out = {"hist_film": list(dict(sorted(rates_96)).values()), "hist_all": list(dict(sorted(rates_all)).values())}

In [125]:
out

{'hist_film': [6, 20, 43, 123, 103],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [126]:
import json
with open("lab01.json", "w") as outfile:
    outfile.write(json.dumps(out))

In [127]:
json.load(open("lab01.json"))

{'hist_film': [6, 20, 43, 123, 103],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [34]:
films.take(1)

[['1',
  'Toy Story (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
  '0',
  '0',
  '0',
  '1',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']]

In [128]:
sc.stop()